<a href="https://colab.research.google.com/github/marizzxxxx/PDI-IFFAKE/blob/main/IFFake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio Pillow facenet-pytorch==2.5.2 torch==2.1.0 opencv-python grad-cam pip install matplotlib

In [53]:
import gradio as gr
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import os
import numpy as np
from PIL import Image
import zipfile
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
with zipfile.ZipFile("//content/drive/MyDrive/PDI/PDI1/examples.zip","r") as zip_ref:
    zip_ref.extractall(".")

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()

model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

checkpoint = torch.load("/content/drive/MyDrive/PDI/PDI2/resnetinceptionv1_epoch_32.pth", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

In [55]:
def predict(input_image:Image.Image, true_label:str):
    """Predict the label of the input_image"""
    face = mtcnn(input_image)
    if face is None:
        raise Exception('No face detected')
    face = face.unsqueeze(0) # add the batch dimension
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)

    # convert the face into a numpy array to be able to plot it
    prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
    prev_face = prev_face.astype('uint8')

    face = face.to(DEVICE)
    face = face.to(torch.float32)
    face = face / 255.0
    face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()

    target_layers=[model.block8.branch1[-1]]
    cam = GradCAM(model=model, target_layers=target_layers)
    targets = [ClassifierOutputTarget(0)]

    grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)
    face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)

    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        prediction = "real" if output.item() < 0.5 else "fake"

        real_prediction = 1 - output.item()
        fake_prediction = output.item()

        confidences = {
            'real': real_prediction,
            'fake': fake_prediction
        }

    return confidences, true_label, face_with_mask

In [56]:
custom_examples = [
    ['/content/drive/MyDrive/PDI/banco de imagens/fake/em-deepfake-no-instagram-mark-zuckerberg-cita-bilhoes-de-dados-roubados-1560354962424_v2_1x1.jpg', 'fake'],
    ['/content/drive/MyDrive/PDI/banco de imagens/real/mark.jpg', 'real'],
    ['/content/drive/MyDrive/PDI/banco de imagens/fake/papaa.png', 'fake'],
    ['/content/drive/MyDrive/PDI/banco de imagens/real/papaaa.png', 'real'],
    # Adicione mais exemplos conforme necessário
]

interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.components.Image(label="Coloque sua imagem aqui", type="pil"),  # Updated component import and type
        gr.components.Text(label="Digite: Real ou fake?")  # Updated component import
    ],
    outputs=[
        gr.components.Label(label="Detector:"),  # Updated component import
        gr.components.Text(label="Você disse que era:"),  # Updated component import
        gr.components.Image(label="Eplicação", type="numpy")  # Updated component import and type
    ],
    examples=custom_examples,  # Substitua pela lista personalizada
    cache_examples=True
).launch()

Caching examples at: '/content/gradio_cached_examples/419'
Caching example 1/4
Caching example 2/4
Caching example 3/4
Caching example 4/4

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://108095443b995a5500.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
